# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv

In [2]:
import dask.dataframe as dd

/var/folders/g5/sfk7tv6168j_mt0b4cpzrm000000gn/T/ipykernel_8552/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
dd_px = dd.read_parquet(parquet_files)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
dd_feat = (dd_px
          .groupby('ticker', group_keys=False)
          .apply(
              lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
          ))

dd_feat = (dd_feat
          .groupby('ticker', group_keys=False)
          .apply(
              lambda x: x.assign(Adj_close_lag_1 = x['Adj Close'].shift(1))
          ).assign(
              returns = lambda x: x['Adj Close']/x['Adj_close_lag_1'] - 1
          ).assign(
              hi_lo_range = lambda x: x['High'] - x['Low']
          ))

/var/folders/g5/sfk7tv6168j_mt0b4cpzrm000000gn/T/ipykernel_8552/423080500.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px
/var/folders/g5/sfk7tv6168j_mt0b4cpzrm000000gn/T/ipykernel_8552/423080500.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_feat


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [5]:
# Write your code below.
df = dd_feat.compute()
df['average'] = df['returns'].rolling(10).mean()

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Answer

It is not necessary to convert the dask dataframe into a pandas dataframe to calculate the moving average. The only advantage of using one or the other in this context is the speed. For comparison, the following code will add the 'average' column in dask and we can compare the time it took for both methods.

In [6]:
# Adding a column of moving average
dd_feat = (dd_feat
          .groupby('ticker', group_keys=False)
          .apply(
              lambda x: x.assign(average = x['returns'].rolling(10).mean())
          ))

/var/folders/g5/sfk7tv6168j_mt0b4cpzrm000000gn/T/ipykernel_8552/2522422337.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_feat


In [7]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_close_lag_1,returns,hi_lo_range,average
ticker,,,,,,,,,,,,,,,
HUM,2001-01-02,14.500000,15.250000,14.312500,14.937500,13.337629,933100,Health Care,Managed Health Care,2001,NaN,NaN,NaN,0.937500,NaN
HUM,2001-01-03,14.687500,15.625000,14.250000,14.437500,12.891182,1077200,Health Care,Managed Health Care,2001,14.937500,13.337629,-0.033473,1.375000,NaN
HUM,2001-01-04,14.375000,14.437500,12.250000,13.187500,11.775060,1217600,Health Care,Managed Health Care,2001,14.437500,12.891182,-0.086580,2.187500,NaN
HUM,2001-01-05,13.000000,14.250000,12.750000,13.625000,12.165699,1606100,Health Care,Managed Health Care,2001,13.187500,11.775060,0.033175,1.500000,NaN
HUM,2001-01-08,13.625000,14.500000,13.312500,13.562500,12.109895,873700,Health Care,Managed Health Care,2001,13.625000,12.165699,-0.004587,1.187500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2021-12-27,34.230000,35.160000,34.220001,35.110001,32.587708,2857400,Information Technology,Communications Equipment,2021,34.130001,31.678108,0.028714,0.939999,0.003317
JNPR,2021-12-28,35.099998,35.549999,35.080002,35.430000,32.884716,3582300,Information Technology,Communications Equipment,2021,35.110001,32.587708,0.009114,0.469997,0.004199
JNPR,2021-12-29,35.450001,35.910000,35.349998,35.770000,33.200291,2243800,Information Technology,Communications Equipment,2021,35.430000,32.884716,0.009596,0.560001,0.006599


## Answer
Calculating the average in dask is slower than doing it in pandas. Therefore it is not better to do it in dask.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.